In [34]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score
from datetime import datetime
from IPython.display import clear_output 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113


In [36]:

train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
#%cd /content/drive/MyDrive/GNN_DATA/ADNI/ALL_NETWORKS
%cd /content/drive/MyDrive/ALL_NETWORKS

for i in range(1, 110):
    if os.path.isfile('AD_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('AD_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp=torch.div(tmp,tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      data = from_networkx(G)
      x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = x
      data.y = 1
      data.edge_attr = data.weight
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

for i in range(1, 110):
    if os.path.isfile('CN_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('CN_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp=torch.div(tmp, tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      data = from_networkx(G)
      x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = x
      data.y = 0
      data.edge_attr = data.weight
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/10M6M6of9sCS0_CWq5d9EF6LhBCK9klNf/ALL_NETWORKS


In [37]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [38]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 25
DataBatch(edge_index=[2, 93592], weight=[93592], num_nodes=2250, x=[2250, 1], y=[25], edge_attr=[93592], batch=[2250], ptr=[26])

Step 2:
Number of graphs in the current batch: 25
DataBatch(edge_index=[2, 91454], weight=[91454], num_nodes=2250, x=[2250, 1], y=[25], edge_attr=[91454], batch=[2250], ptr=[26])

Step 3:
Number of graphs in the current batch: 25
DataBatch(edge_index=[2, 94574], weight=[94574], num_nodes=2250, x=[2250, 1], y=[25], edge_attr=[94574], batch=[2250], ptr=[26])

Step 4:
Number of graphs in the current batch: 25
DataBatch(edge_index=[2, 93856], weight=[93856], num_nodes=2250, x=[2250, 1], y=[25], edge_attr=[93856], batch=[2250], ptr=[26])

Step 5:
Number of graphs in the current batch: 25
DataBatch(edge_index=[2, 97580], weight=[97580], num_nodes=2250, x=[2250, 1], y=[25], edge_attr=[97580], batch=[2250], ptr=[26])

Step 6:
Number of graphs in the current batch: 25
DataBatch(edge_index=[2, 97686], weight=[97686], n

In [39]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GraphConv(in_channels=1, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch       
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [40]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7046
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.7202
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7077
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.7909
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6730
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6863
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7018
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6942
Epoch: 009, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6927
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7012
Epoch: 011, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6958
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6978
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7036
Epoch: 014, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6973
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6992
Epoch: 016, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.7116
Epoch: 0

In [41]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GCNConv, JumpingKnowledge, global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.num_layers  = 3
        self.hidden = 16
        self.conv1 = GCNConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(GCNConv(self.hidden, self.hidden))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index,weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, weight))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [42]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6863
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6726
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6950
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6928
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6932
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6940
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6943
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6928
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7042
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6965
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6931
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6931
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6931
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6932
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6934
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6916
Epoch: 0

In [43]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(1, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [44]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6804
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.5767
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6430
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6700
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6850
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6395
Epoch: 007, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6708
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.5385, Loss: 0.5924
Epoch: 009, Train Acc: 0.5867, Test Acc: 0.4615, Loss: 0.6562
Epoch: 010, Train Acc: 0.6200, Test Acc: 0.5000, Loss: 0.6039
Epoch: 011, Train Acc: 0.6000, Test Acc: 0.4808, Loss: 0.6124
Epoch: 012, Train Acc: 0.6200, Test Acc: 0.5000, Loss: 0.6577
Epoch: 013, Train Acc: 0.6400, Test Acc: 0.5000, Loss: 0.6970
Epoch: 014, Train Acc: 0.6200, Test Acc: 0.5000, Loss: 0.7612
Epoch: 015, Train Acc: 0.6267, Test Acc: 0.5000, Loss: 0.7834
Epoch: 016, Train Acc: 0.6267, Test Acc: 0.5192, Loss: 0.6308
Epoch: 0

In [45]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [46]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 1000):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6955
Epoch: 002, Train Acc: 0.4867, Test Acc: 0.4615, Loss: 0.6935
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6935
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6938
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6913
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6915
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6912
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7014
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6941
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6904
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6950
Epoch: 012, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6933
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6980
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6918
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6937
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6922
Epoch: 0

In [47]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [48]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7295
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6942
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6907
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6965
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6903
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6930
Epoch: 007, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6956
Epoch: 008, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6929
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6930
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6951
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6927
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6988
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6925
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6935
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6940
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6950
Epoch: 0

In [49]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [50]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6954
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6929
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6989
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6914
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6923
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.4808, Loss: 0.6929
Epoch: 007, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6930
Epoch: 008, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.7209
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6924
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6849
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6924
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6945
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7066
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6947
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6987
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6926
Epoch: 0

In [51]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [52]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.7393
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7211
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6964
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6932
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6934
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6955
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6936
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6931
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6928
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6928
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6931
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6925
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6971
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6935
Epoch: 015, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6938
Epoch: 016, Train Acc: 0.4667, Test Acc: 0.4808, Loss: 0.6935
Epoch: 0

In [53]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [54]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6935


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch: 002, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6940
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6940
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6927
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7032
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6926
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6940
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6965
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6915
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6929
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6962
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6953
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6919
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6937
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6922
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6937
Epoch: 017, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6923
Epoch: 0

In [55]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [56]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.Set2Set' is deprecated, use 'nn.aggr.Set2Set' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7000
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6883
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6924
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6904
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7011
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6907
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6944
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6997
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6906
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6923
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6935
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6957
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6966
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6926
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6927
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6964
Epoch: 0

In [57]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [58]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7102
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6909
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6953
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6871
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6874
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6942
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6926
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6946
Epoch: 009, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6940
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6925
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6878
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6926
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6898
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6940
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6906
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6919
Epoch: 0

In [59]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [61]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6925
Epoch: 002, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6933
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6930
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6926
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6924
Epoch: 006, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6984
Epoch: 007, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6916
Epoch: 008, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.7006
Epoch: 009, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6935
Epoch: 010, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6943
Epoch: 011, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6925
Epoch: 012, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6945
Epoch: 013, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6939
Epoch: 014, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6943
Epoch: 015, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6926
Epoch: 016, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6925
Epoch: 0